## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import sys
sys.path.insert(0, '..')

In [3]:
from loguru import logger

In [4]:
import json

from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from tqdm.notebook import tqdm

import nest_asyncio
nest_asyncio.apply()

## Setup vector store

In [6]:
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.opensearch import (
    OpensearchVectorClient,
    OpensearchVectorStore,
)
from llama_index.core import VectorStoreIndex, StorageContext


In [7]:
endpoint = os.getenv('OPENSEARCH_ENDPOINT', 'http://localhost:9200')
index = os.getenv('OPENSEARCH_INDEX', 'index-demo')

## Load document

In [8]:
from datasets import load_dataset

DATA_PATH = os.getenv('DATA_PATH', 'data')
DATASET_NAME = "BookingCare/ViHealth-NLI"

datasets = load_dataset(DATASET_NAME, split='test')
datasets.save_to_disk(os.path.join(DATA_PATH, DATASET_NAME))

Saving the dataset (0/1 shards):   0%|          | 0/899 [00:00<?, ? examples/s]

In [9]:
datasets

Dataset({
    features: ['id', 'sentence1', 'sentence2'],
    num_rows: 899
})

In [10]:
from llama_index.core import Document
documents = []
for document in datasets:
    text = document['sentence1'] + '.\n'+ document['sentence2']
    doc = Document(
        text=text
    )
    documents.append(doc)
logger.info(f"[COLLECT] {len(documents)=}")

2024-11-17 00:39:39.270 | INFO     | __main__:<module>:9 - [COLLECT] len(documents)=899


In [11]:
from llama_index.core.schema import MetadataMode

document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

Rau xanh lá đậm.
Các loại rau có màu xanh lá đậm như cải bó xôi, bông cải xanh, mồng tơi, cải xoăn... chứa hàm lượng cao folate và một loại vitamin B được chứng minh giúp cải thiện khả năng rụng trứng ở phụ nữ .


## Setting LLM

In [12]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [13]:
model_name = 'BookingCare/gte-multilingual-base-v1'
embed_model = HuggingFaceEmbedding(model_name, device='cpu', trust_remote_code=True)

In [14]:
from llama_index.core import Settings
Settings.embed_model = embed_model

## Vector Store

In [15]:
DIM = 768
embedding_field = "embedding"
client = OpensearchVectorClient(
    endpoint=endpoint,
    index=index,
    dim=DIM,
    embedding_field=embedding_field
)

In [16]:
vector_store = OpensearchVectorStore(client)

## Index embeddings

In [17]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SentenceSplitter

In [18]:
chunker = SentenceSplitter

chunker_cfg = {
    "chunk_size": 1024,
    "chunk_overlap": 10,
}

In [20]:
t0 = time.perf_counter()
logger.info(f'Creating new DB index')
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

pipeline = IngestionPipeline(
    transformations=[
        chunker(**chunker_cfg),
        embed_model,
    ],
    vector_store=vector_store
)
num_workers=None
# num_workers = os.cpu_count() - 1
# os.environ['TOKENIZERS_PARALLELISM'] = 'true'
# set_start_method('spawn', force=True)
logger.info(f'Running Ingestion Pipeline with {num_workers=}...')
nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)
docstore = SimpleDocumentStore()
await docstore.async_add_documents(nodes)
storage_context = StorageContext.from_defaults(
    docstore=docstore,
    vector_store=vector_store
)
logger.info(f'Persisting Storage Context to: ')
storage_context.persist(os.path.join(DATA_PATH, 'storage_context'))
t1 = time.perf_counter()

2024-11-17 00:44:28.380 | INFO     | __main__:<module>:2 - Creating new DB index
2024-11-17 00:44:28.381 | INFO     | __main__:<module>:17 - Running Ingestion Pipeline with num_workers=None...


Parsing nodes:   0%|          | 0/899 [00:00<?, ?it/s]

Generating embeddings: 100%|████| 109/109 [04:07<00:00,  2.27s/it]
2024-11-17 00:48:38.659 | INFO     | __main__:<module>:25 - Persisting Storage Context to: 


In [21]:
logger.info(f'Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s')
logger.info(f'[COLLECT] {len(nodes)=}')

2024-11-17 00:49:58.416 | INFO     | __main__:<module>:1 - Indexing 899 documents into VectorStoreIndex took 251s
2024-11-17 00:49:58.417 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=1085


In [22]:
for i, node in enumerate(nodes[:2]):
    print(f"\n\n=======Node{i + 1}===========")
    print(node.get_text())



=======Node1===========
Rau xanh lá đậm.
Các loại rau có màu xanh lá đậm như cải bó xôi, bông cải xanh, mồng tơi, cải xoăn... chứa hàm lượng cao folate và một loại vitamin B được chứng minh giúp cải thiện khả năng rụng trứng ở phụ nữ .


=======Node2===========
Sơ cứu trật khớp ban đầu.
Sau đây là cách sơ cứu trật khớp đơn giản bạn nên biết: Không di chuyển khớp, không nắn hoặc cố cử động khớp bị trật. Nghi ngờ gãy xương hoặc sai khớp cột sống: Tránh xoay vặn thân mình, nằm ngửa trên ván cứng, chèn chắc 2 bên cổ và thân mình bằng bao cát hoặc chăn gối. Sơ cứu trật khớp vai: Sử dụng dây vải hoặc dây băng treo cẳng tay vào cổ. Sơ cứu trật khớp khuỷu: cố định bằng 2 nẹp trước sau có độn bông. Sơ cứu trật khớp háng: cố định như gãy xương đùi, nằm ngửa, kê gối và chèn cho bệnh nhân trong tư thế hiện có, nhanh chóng chuyển bệnh nhân đến cơ sở y tế nắn chỉnh hình. Việc cấp cứu ban đầu gãy xương, trật khớp, bong gân rất quan trọng. Tuy nhiên, nếu không thực hiện sơ cứu đúng cách thì rất có t